In [510]:
import pandas as pd 
import numpy as np
import locale

In [511]:
raw=pd.read_csv('C:\\Users\\srikrishna.v\\Desktop\\DSL Bootcamp\\RAw.csv')

In [512]:
raw.head()

,order_date,buyer_org_id,seller_org_id,order_status,order_id,order_line_id,category,rt_mis_category,vertical,brand,total_line_amount
0,2020-09-13,ORGEH09Y6R25ZCD6F1F10LM46B22T,ORG6XT7F3J75M8T3Z5FYNMEDRT4YR,SELLER_ORDER_PAYMENT_RECEIVED,ODEH0489734M8T,OL6e6aeee6cf8246f684c5d65318f62c34,Acc & CE,Acc & CE,CameraBattery,"""",16.00
1,2020-09-13,ORGL7F6FXV68FQ9VFPJGQNDPXSM8M,ORGGMM56LYQ8SQK5FT27QEHZSXH8P,SELLER_ORDER_PAYMENT_RECEIVED,ODPJG159945HZS,OL00ce99f27bec447982b11aa20d31c55b,Food-FMCG,FMCG,AntisepticandHygieneLiquid,Suthol,431.99
2,2020-09-13,ORGFEQXGY357SQ09FMJ16372SLR63,ORG73FW3EL0ECDFK4LQSPSQ46CNLY,SELLER_ORDER_PAYMENT_RECEIVED,ODLR69997876CN,OL47e31de286d74eba9d98919c07222bee,Clothing,Clothing,Blazers,Ennoble,1703.52
3,2020-09-13,ORG0V5T91KJVN8VX4W9X1335QN7E9,ORGZ72571P70T8KBZEK9DZW830QXZ,SELLER_ORDER_PAYMENT_RECEIVED,ODN7E599936EK9,OLc3d2cb7b1f364a1f9ee4719a0562feda,Clothing,Clothing,Briefs,MACROMAN,1134.00
4,2020-09-13,ORGZP04WLE5YQDH346L56DB99KGQB,ORG0QNP19992MCXQGEQH4VDL72632,SELLER_ORDER_PAYMENT_RECEIVED,ODKGQ000057QNP,OL4e7877a844f0411fb6117f80ac060cb9,Clothing,Clothing,ClothingBlouse,F TEX India,1130.53


In [513]:
raw['category'].fillna('Blank',inplace=True)
raw['bu']=raw['category'].apply(bu_map)
raw['bg']=raw['category'].apply(bg_map)
raw['overall']= 'overall' #adding a handle , to get overall aggregate  later on 
raw['include']=1 #another handle , to prevent the adjustments from reflecting in overalls
raw.shape

(167933, 15)

In [514]:
#Filtering out Unusually High GMV orders 
high=raw.groupby('order_id',as_index=False).agg(gmv=('total_line_amount',np.sum))
high=high[high.gmv>200000].order_id
raw=raw[~raw.order_id.isin(high)]


In [515]:
#Fetch Saffola Orders for Seperate Treatment
saf=raw[ (raw['brand']=='Saffola') & (raw['rt_mis_category']=='Sugar & Oil')]
saf=saf.copy()
#We are avoiding the SettingWithCopyWarning by creating a copy
saf['rt_mis_category']='FMCG'
saf['include']=0


In [516]:
#Fetch  ELHW orders for Seperate Treatment
ELHW_orgs=['ORGZ2F605Z1HMQK4FW5E9TDGRDN16', 'ORGE9XKRM3M5TC31FK94J8CW9SNR8' , 'ORG1YC0NB2EHQC68ZNR6FM9E716WV' , 'ORGB8DXD3P3VCDTWFDNW7Z1GWW3H1','ORG7FE5ZKM9SVDRXGTBD61DT844WN' ,'ORG4E57V64T05882ZVSX0PYGSRVB5' , 'ORG6BS0X7S99TCQ8GE9GKRHV08E8L' , 'ORG6XEYZSH5N5DF1FJ78KT7JV7V39' ,'ORG2W3YRFFXGJDKH43B5B84ELLLS9', 'ORGK3JLY50YFMQ2JZ5YBS8LS0SWXB','ORG1ZDLV1EKVPQC3ZJCRWNFLP95L4','ORG1ZDLV1EKVPQC3ZJCRWNFLP95L4','ORGK4YQ8PFK5VQY1ZBGYQX62YBWQ1']
ELHW_verts=["Soap Dispensers","CircuitBreakers","DoorFittingsKit","Bathroom Shelve","Toilet Paper Holder","Drain Cover","Towel Holder","Soap Dish and Tumbler Holder","Hooks ","Faucets & Taps","Wall and Sink Mixer","Shower","Health Faucets","Aldrop","Latch","Handles","Tower Bolt","Door Stopper","Door Fittings Kit","Door Closer","Hinge","Door Eye","Mortise Locks","Rim Locks","Pad Locks","Cabinet Locks","Door Knobs/Cyllindrical Locks","Cyllinder Locks","Glass Lock","Ledbulb","Ledpanellight","Ledstring","LEDTubelight","Emergencylight","Ledlantern","TubeHolders","BulbHolder","SwitchesandIndicators","Sockets","SwitchPlates","SwitchSocketCombination","MountingBox","DoorChimesandBells","PlugsandPins","RegulatorandDimmer","ExtensionCords","Circuit Breaker","Fans","TowerBolt","Faucet","ElectricalWire"]
elhw=raw[(raw.seller_org_id.isin(ELHW_orgs)) & (raw.vertical.isin(ELHW_verts)) & ~raw.rt_mis_category.isin(ELHW)]
elhw.head()
#We are avoiding the SettingWithCopyWarning by creating a copy
elhw=elhw.copy()
elhw.rt_mis_category='Electrical'
elhw.include=0



In [517]:
#Adding the additional records to be processed to the dataframe
raw=pd.concat([raw,saf,elhw])
raw.shape

(168088, 15)

In [518]:
DAY=raw.order_date.max()
DAY
#Todays Numbers

'2020-09-13'

In [519]:
#We are better off doing this in SQL while dataset preparation itself
Food=['FMCG','Core Staples','Other Staples','Fresh','Sugar & Oil','Meat']
Pharma=['Pharma']
Electronics=['Featurephone','LandlinePhones','Refurbishedmobiles','Smartphone','Acc & CE','IT']
ELHW=['Electrical']
GM=['Home & Kitchen','Toys','Luggage & Bags']
LS=['Clothing','Footwear']
def bg_map(s):
      if s in Food:
          return 'Food'
      elif s in Pharma:
          return 'Pharma'
      else:
          return 'Mario'
def bu_map(s):
      if s in Electronics:
          return 'Electronics'
      elif s in ELHW:
          return 'Hardware & Electrical'
      elif s in LS:
          return 'Lifestyle'
      elif s in GM:
          return 'General Merchandise'
      else:
          return 'Redundant'
raw.category.unique()    


array(['Acc & CE', 'Food-FMCG', 'Clothing', 'H&K-Non Metals', 'Pharma',
       'Featurephone', 'H&K-Metals', 'Smartphone', 'Toys & Baby Care',
       'Fresh', 'Footwear', 'Electrical', 'Luggage',
       'Fullfilment Material', 'Blank', 'IT', 'Refurbishedmobiles'],
      dtype=object)

In [520]:
cat=raw[raw.order_date==DAY].groupby('category', as_index=False).agg(buyers=('buyer_org_id',pd.Series.nunique),orders=('order_id',pd.Series.nunique),gmv=('total_line_amount',np.sum))
#This is awesome 
overall=raw[(raw.order_date==DAY)&(raw.include==1)].groupby('overall', as_index=False).agg(buyers=('buyer_org_id',pd.Series.nunique), orders=('order_id', pd.Series.nunique), gmv=('total_line_amount',np.sum))
bu=raw[(raw.order_date==DAY)&(raw.include==1)].groupby('bu',as_index=False).agg(buyers=('buyer_org_id', pd.Series.nunique), orders=('order_id',pd.Series.nunique), gmv=('total_line_amount', np.sum))
bg=raw[(raw.order_date==DAY)&(raw.include==1) ].groupby('bg',as_index=False).agg(buyers=('buyer_org_id', pd.Series.nunique), orders=('order_id',pd.Series.nunique), gmv=('total_line_amount', np.sum))

cat.rename(columns={'category':'entity'}, inplace=True)
bu.rename(columns={'bu':'entity'}, inplace=True)
bg.rename(columns={'bg':'entity'}, inplace=True)
overall.rename(columns={'overall':'entity'}, inplace=True)


In [529]:
#merge all the processed data
df=pd.concat([overall, bu,bg,cat],ignore_index=True)
df['AOV']=df.gmv/df.orders
df['ABS']=df.gmv/df.buyers
locale.setlocale(locale.LC_MONETARY, 'en_IN')
df.gmv=df.gmv.apply(lambda x:locale.currency(x,symbol=False,grouping=True).split('.')[0])
df.buyers=df.buyers.apply(lambda x:locale.currency(x,symbol=False,grouping=True).split('.')[0])
df.orders=df.orders.apply(lambda x:locale.currency(x,symbol=False,grouping=True).split('.')[0])
df.AOV=df.AOV.apply(lambda x:locale.currency(x,symbol=False,grouping=True).split('.')[0])
df.ABS=df.ABS.apply(lambda x:locale.currency(x,symbol=False,grouping=True).split('.')[0])

df



,entity,buyers,orders,gmv,AOV,ABS
0,overall,"26,361","61,657","19,26,57,010","3,124","7,308"
1,Electronics,"8,249","13,403","7,85,98,726","5,864","9,528"
2,Hardware & Electrical,46,51,"2,29,124","4,492","4,980"
3,Lifestyle,"2,910","4,391","2,36,10,090","5,376","8,113"
4,Redundant,"15,407","43,894","9,02,19,069","2,055","5,855"
5,Food,"2,183","3,443","34,19,085",993,"1,566"
6,Mario,"22,208","55,398","18,30,27,545","3,303","8,241"
7,Pharma,"2,326","2,816","62,10,380","2,205","2,669"
8,Acc & CE,"5,820","8,556","2,29,05,586","2,677","3,935"
9,Blank,218,254,"4,72,620","1,860","2,167"
